<a href="https://colab.research.google.com/github/coldsober-irene/NLP-fellowship/blob/main/Fabien_Mbonigaba_Hackathon1_File.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi pyngrok uvicorn nest-asyncio

In [ ]:
!pip install -U easynmt

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from fastapi import FastAPI, Response
from pyngrok import ngrok
import nest_asyncio
import uvicorn
from starlette.responses import HTMLResponse
import json
from easynmt import EasyNMT

In [ ]:
# set authentication for ngrok
!ngrok authtoken 2HJQ5KFUMPejTBPPFOQGP2WRU74_o2vYuu2ZQcGb3zYD6ziq

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# SOUP
link = 'https://www.jobinrwanda.com/jobs/featured'
soup = BeautifulSoup(requests.get(link).content, 'html.parser')

In [ ]:
# LINKS OF ALL FEATURED CONTENT ON WEB
# job title
job_titles = {}

divs = soup.find_all('div', class_ = 'card-body p-2')
f_links = []

for index,div in enumerate(divs):
  anchor = div.find('a')
  found_link = 'https://www.jobinrwanda.com'+anchor['href']
  job_titles[index] = anchor.find('h5', class_ = 'card-title').get_text()
  f_links.append(found_link)
job_titles = list(job_titles.values())

In [ ]:
job_titles

In [ ]:
# EACH LINK SOUP
l_soups = {}
for index,link in enumerate(f_links):
  SOUP = BeautifulSoup(requests.get(link).content, 'html.parser')
  l_soups[index] = SOUP

In [ ]:
# DIVS FOR EACH FEATURE CONTENT
content_divs = {}
job_info = {}
for index,soup1 in enumerate(list(l_soups.values())):
  content_divs[index] = soup1.find('div', class_ = 'clearfix text-formatted field field--name-field-job-full-description field--type-text-long field--label-hidden field__item')
  ul = soup1.find('ul', class_ = 'list-group list-group-flush')
  lists = {}
  for li in ul.children:
    try:
      info = re.sub('\s{2,}|\n',' ',li.text) # both key and value together as str
      # separate key & value to get key:value dictionary
      if not 'apply' in info.lower():
        if 'viewed' in info:
          times = re.search('\d+', info).group()
          lists['views'] = times
        else:
          k,value = re.split(':', info)[:2]
          lists[k.lstrip()] = value
    except AttributeError:
      pass
  job_info[index] = lists


In [ ]:
# FULL CONTENT OF EACH FEATURE
full_content = {}
for index,div in enumerate(list(content_divs.values())):
  one_div_content = []
  for para in div.find_all('p', class_ = 'text-align-justify'):
    one_div_content.append(para.get_text())
  full_content[index] = re.sub('\n|\\xa0','','\n'.join(one_div_content))

In [ ]:
final = []
for index in list(content_divs.keys()):
  index_info = list(job_info[index].values())
  index_text = [full_content[index]]
  final.append(index_info + index_text)
final


In [ ]:
columns = list(job_info[0].keys()) + ['Description']
len(columns)

In [ ]:
dataframe = pd.DataFrame(final, columns = columns)

In [ ]:
dataframe['job title'] = job_titles

In [ ]:
dataframe['job title'] = dataframe['job title'].str.replace('\n','',regex=True).astype('str')

In [ ]:
only_IT_jobs = dataframe.copy()
only_IT_jobs['is IT job'] = False

for index in range(len(only_IT_jobs)):
  if re.findall('Computer|IT|Software', only_IT_jobs.iloc[index]['Sector']):
    only_IT_jobs['is IT job'][index] = True

In [ ]:
only_IT_jobs = only_IT_jobs[only_IT_jobs['is IT job'] == True]   # filter for the IT job

# umucyo webscrapping

In [ ]:
url = 'https://www.umucyo.gov.rw/eb/bav/selectListAdvertisingListForGU.do'
umucyo_soup = BeautifulSoup(requests.get(url).content, 'html.parser')
table = umucyo_soup.find('table', class_ = 'article_table mb10')

In [ ]:
head_row = table.find('thead').find('tr')
titles = []
for th in head_row.find_all('th'):
  titles.append(re.sub('\r|\n|\t','',th.text))
titles = titles[1:]

In [ ]:
# table body
body_rows = table.find('tbody').find_all('tr') 
full_data = []
for row in body_rows:
  ls = []
  for detail in row.find_all('td'):
    ls.append(re.sub('\n|\r|\t|\s', '',detail.text))
  full_data.append(ls[1:])


In [ ]:
umucyo_df = pd.DataFrame(full_data, columns = titles)

#dataframe translation

In [ ]:
model = EasyNMT('opus-mt')
app = FastAPI(title = 'my application api')
prev = 'en'
@app.get('/let')
def do(lang:str='', orient:str='index'):
  global prev
  try:
    for index in range(len(only_IT_jobs)):
      only_IT_jobs['job title'][index] = model.translate(only_IT_jobs.iloc[index]['job title'], source_lang = prev, target_lang = lang)
      only_IT_jobs['Description'][index] = model.translate(only_IT_jobs.iloc[index]['Description'], source_lang = prev, target_lang = lang)
      prev = str(lang)
    return Response(only_IT_jobs.to_json(orient = orient), media_type = 'application/json')
  except:
    pass

In [ ]:
# show all IT jobs
@app.get('/itjobs')
def show(orient:str='index'):
    return Response(only_IT_jobs.to_json(orient = orient), media_type = 'application/json')

In [ ]:
# show all information from jobinrwanda
@app.get('/jobinrwanda_all')
def showall(orient:str='index'):
    return Response(dataframe.to_json(orient = orient), media_type = 'application/json')

In [ ]:
# show all information retrieved from umucyo
@app.get('/umucyo_all')
def show_umucyo(orient:str='index'):
    return Response(umucyo_df.to_json(orient = orient), media_type = 'application/json')

In [ ]:
tunnel = ngrok.connect(8000)
print("PUBLIC URL: ", tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port = 8000)